In [125]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import glob
import os




In [90]:
# path = '/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/restaurant_data' # use your path
# joined_files = os.path.join(path, "data_breakfast_club*.csv")


In [91]:
# df = pd.concat(
#     map(pd.read_csv, ['data_breakfast_club.csv', 'data_dishoom.csv', 'duck-and-waffle-london.csv', 'flat-iron-london-2.csv', 'mother-mash-london.csv']), ignore_index=True)
# print(df)

In [147]:
df = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/restaurant_data/data_dishoom.csv', header = None)
# df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/test_data/test.csv', header=None)
# df_test = df.rename({0: 'label', 1: 'text'}, axis = 1)
df = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [148]:
df = df['text']

In [149]:
df = pd.DataFrame(df)

In [150]:
df.head()

,text
0,Review
1,Hundreds of happy customers come through every...
2,We came for breakfast on a Monday so we were a...
3,Have been wanting to go to Dishoom for years a...
4,The best Indian food in London!This is my 3rd ...


In [151]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

In [152]:
df_text = df.text

In [153]:
df_text = pd.DataFrame(df_text)

In [154]:
df_text

,text
0,Review
1,Hundreds of happy customers come through every...
2,We came for breakfast on a Monday so we were a...
3,Have been wanting to go to Dishoom for years a...
4,The best Indian food in London!This is my 3rd ...
...,...
496,"We waited 51 minutes, but was quoted 1 hour 10..."
497,Had a fantastic experience at Dishoom Covent G...
498,I tried to go for Dishoom for dinner one night...
499,"The lamp chops are awesome. It's tender, a bit..."


In [155]:
len_train = int(10/100*len(df_text['text']))
df_text = df_text['text']
df_text = df_text[:len_train]

In [156]:
df_text = pd.DataFrame(df_text)

In [157]:
df_text['clean_text'] = df_text.text.apply(clean)
df_text['clean_text'] = df_text['clean_text'].astype('str')

In [158]:
df_text

,text,clean_text
0,Review,['review']
1,Hundreds of happy customers come through every...,"['hundred', 'happy', 'customer', 'come', 'ever..."
2,We came for breakfast on a Monday so we were a...,"['came', 'breakfast', 'monday', 'able', 'bypas..."
3,Have been wanting to go to Dishoom for years a...,"['wanting', 'go', 'dishoom', 'year', 'glad', '..."
4,The best Indian food in London!This is my 3rd ...,"['best', 'indian', 'food', 'london', 'visit', ..."
5,I'm sure there's not much more I could say bec...,"['sure', 'much', 'could', 'say', 'dishoom', 'f..."
6,4.5/5 stars As someone who isn't well versed i...,"['star', 'someone', 'well', 'versed', 'indian'..."
7,Food 5/5Ambiance 5/5Service 5/5You'll likely h...,"['food', 'likely', 'wait', 'anywhere', 'minute..."
8,It is a chain restaurant but nothing about it ...,"['chain', 'restaurant', 'nothing', 'lack', 'qu..."
9,Wow wow wow.. A very memorable experience and ...,"['wow', 'wow', 'wow', 'memorable', 'experience..."


In [164]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(df_text['clean_text'])

lda_model = LatentDirichletAllocation(n_components=2)

lda_vectors = lda_model.fit_transform(data_vectorized)

In [165]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [166]:
print_topics(lda_model, vectorizer)

Topic 0:
[('chicken', 25.868909247962268), ('dish', 18.660490207588616), ('restaurant', 16.430469320987804), ('came', 13.056048037688248), ('garlic', 12.519085624891499), ('chai', 11.581295653428521), ('food', 11.136662210146952), ('naan', 11.00902149678948), ('house', 10.56391141870383), ('delicious', 10.551462791586054)]
Topic 1:
[('chicken', 39.13109075203666), ('food', 30.863337789851986), ('london', 26.44039870377664), ('dishoom', 25.093425945185526), ('naan', 21.990978503209465), ('restaurant', 19.569530679011134), ('dish', 19.339509792410322), ('get', 18.125798527706618), ('great', 16.64932426743048), ('back', 16.370794648904255)]
